# Download Agent Demo

This notebook demonstrates the Download Agent functionality.

The Download Agent provides:
- Smart URL handling (detects landing pages)
- Context-aware file naming
- Special handling for GitHub, HuggingFace, etc.
- Multiple download support
- File listing

## Setup

In [7]:
from aw_agents import djoin 
djoin.rootdir

'/Users/thorwhalen/.local/share/aw/agents'

In [ ]:
# Import the agent
from aw_agents.agents.download import DownloadAgent
from pathlib import Path
import tempfile

# Create a temporary directory for this demo
demo_dir = Path(djoin("download_agent_demo"))
print(f"Demo downloads will go to: {demo_dir}")

# Initialize the agent with our demo directory
agent = DownloadAgent(default_download_dir=demo_dir)

Demo downloads will go to: /Users/thorwhalen/.local/share/aw/agents/download_agent_demo


## Explore Available Tools

Let's see what tools the agent provides:

In [9]:
# Get tools
tools = agent.get_tools()

print(f"Available tools ({len(tools)}):")
print()

for tool in tools:
    print(f"📌 {tool['name']}")
    print(f"   {tool['description'][:100]}...")
    print()

Available tools (3):

📌 download_content
   Download content from a URL with intelligent handling. Automatically detects landing pages and finds...

📌 download_multiple
   Download multiple URLs at once. Returns paths to all downloaded files. Each URL can have its own con...

📌 list_downloads
   List files in the downloads directory. Helps track what has been downloaded....



## Test 1: Download a Simple Text File

Let's start with something simple - a small text file.

In [ ]:
# Download a simple text file from a known source
result = agent.execute_tool('download_content', {
    'url': 'https://www.w3.org/History/19921103-hypertext/hypertext/README.html',
    'context': 'WWW History Document'
})

print(f"Success: {result['success']}")
print(f"Message: {result['message']}")
if result['success']:
    print(f"\nDownloaded to: {result['data']['path']}")
    print(f"Final URL: {result['data']['url']}")
    if result.get('warnings'):
        print(f"\nWarnings: {result['warnings']}")

## Test 2: GitHub URL Handling

The agent automatically converts GitHub blob URLs to raw URLs for direct download.

In [ ]:
# Example with a GitHub blob URL (if you have one, replace with a real URL)
# This demonstrates the automatic conversion

github_example = "https://github.com/thorwhalen/graze/blob/master/README.md"

result = agent.execute_tool('download_content', {
    'url': github_example,
    'context': 'Graze README'
})

print(f"Success: {result['success']}")
if result['success']:
    print(f"Message: {result['message']}")
    print(f"\nOriginal URL: {github_example}")
    print(f"Downloaded from: {result['data']['url']}")
    print(f"Saved to: {result['data']['path']}")
    print(f"\nNote: The blob URL was automatically converted to raw URL!")
else:
    print(f"Error: {result['message']}")

## Test 3: Context-Aware File Naming

Watch how the context becomes a meaningful filename:

In [ ]:
# Download the same URL with different contexts to see filename generation
test_url = "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf"

contexts = [
    "Test PDF Document",
    "Accessibility Test File",
    "W3C Sample PDF"
]

for context in contexts:
    result = agent.execute_tool('download_content', {
        'url': test_url,
        'context': context
    })
    
    if result['success']:
        filename = Path(result['data']['path']).name
        print(f"Context: '{context}'")
        print(f"  → Filename: {filename}")
        print()

## Test 4: Download Multiple Files

Download several files at once:

In [ ]:
# Multiple files with contexts
urls = [
    "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf",
    "https://www.w3.org/History/19921103-hypertext/hypertext/README.html",
]

contexts = [
    "Test PDF for Demo",
    "W3C History Document",
]

result = agent.execute_tool('download_multiple', {
    'urls': urls,
    'contexts': contexts
})

print(f"Success: {result['success']}")
print(f"Message: {result['message']}")
print()

if result['success']:
    print(f"Results:")
    print(f"  Total: {result['data']['total']}")
    print(f"  Successful: {result['data']['successful']}")
    print(f"  Failed: {result['data']['failed']}")
    print()
    
    for i, file_result in enumerate(result['data']['results'], 1):
        print(f"{i}. {Path(file_result['path']).name if file_result['path'] else 'FAILED'}")
        if not file_result['path'] and file_result.get('warnings'):
            print(f"   Warnings: {file_result['warnings']}")

## Test 5: List Downloaded Files

See what we've downloaded so far:

In [ ]:
# List all files
result = agent.execute_tool('list_downloads', {})

print(f"Success: {result['success']}")
print(f"Message: {result['message']}")
print()

if result['success']:
    print(f"Directory: {result['data']['directory']}")
    print(f"Total files: {result['data']['total']}")
    print()
    print("Files:")
    for file_info in result['data']['files']:
        print(f"  📄 {file_info['name']}")
        print(f"     Size: {file_info['size_formatted']}")
        print()

## Test 6: List Only PDFs

In [ ]:
# List only PDF files
result = agent.execute_tool('list_downloads', {
    'pattern': '*.pdf'
})

print(f"Success: {result['success']}")
print(f"Message: {result['message']}")
print()

if result['success']:
    print(f"PDF files found: {result['data']['total']}")
    for file_info in result['data']['files']:
        print(f"  📕 {file_info['name']} ({file_info['size_formatted']})")

## Test 7: Agent Metadata

In [ ]:
# Get agent metadata
metadata = agent.get_metadata()

print("Agent Metadata:")
for key, value in metadata.items():
    print(f"  {key}: {value}")

## Test 8: Error Handling

Let's test how the agent handles errors:

In [ ]:
# Try an invalid URL
result = agent.execute_tool('download_content', {
    'url': 'https://this-is-definitely-not-a-real-url-12345.com/fake.pdf',
    'context': 'Nonexistent File'
})

print(f"Success: {result['success']}")
print(f"Message: {result['message']}")
print()
print("✓ Agent handled the error gracefully!")

In [ ]:
# Try an unknown tool
result = agent.execute_tool('nonexistent_tool', {})

print(f"Success: {result['success']}")
print(f"Message: {result['message']}")
print()
print("✓ Agent handled unknown tool gracefully!")

## Summary

The Download Agent provides:

✅ **Smart URL handling** - Detects and converts special URLs (GitHub, HuggingFace)  
✅ **Context-aware naming** - Uses your descriptions for meaningful filenames  
✅ **Multiple downloads** - Download several files at once  
✅ **File listing** - Track what you've downloaded with pattern matching  
✅ **Error handling** - Graceful failures with informative messages  
✅ **Landing page detection** - Finds actual download links automatically  

Next steps:
- Deploy to Claude Desktop using the MCP adapter
- Deploy to ChatGPT using the OpenAPI adapter
- See the main README for integration guides

## Cleanup

Remove the demo directory if you want:

In [ ]:
import shutil

# Uncomment to clean up
# shutil.rmtree(demo_dir)
# print(f"Removed demo directory: {demo_dir}")

print(f"Demo directory still exists at: {demo_dir}")
print("You can explore the downloaded files there.")